In [ ]:
!git clone https://github.com/i-need-sleep/referee.git

Cloning into 'referee'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 40 (delta 4), reused 27 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 35.04 KiB | 588.00 KiB/s, done.
Resolving deltas: 100% (4/4), done.


In [ ]:
%cd referee/code

/content/referee/code


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
    Uninstalling triton-3.1.0:
      Successfully uninstalled triton-3.1.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.21.5
    Uninstalling nvidia-n

In [ ]:
%cd utils

/content/referee/code/utils


In [ ]:
!python fetch_huggingface_models.py

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
tokenizer_config.json: 100% 52.0/52.0 [00:00<00:00, 206kB/s]
config.json: 100% 579/579 [00:00<00:00, 2.70MB/s]
tokenizer_config.json: 100% 52.0/52.0 [00:00<00:00, 221kB/s]
spm.model: 100% 2.46M/2.46M [00:00<00:00, 40.2MB/s]
tokenizer_config.json: 100% 52.0/52.0 [00:00<00:00, 263kB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can 

In [ ]:
%cd ..

/content/referee/code


In [ ]:
import pandas as pd
import torch
from models.deberta_for_eval import DebertaForEval
import utils.globals as uglobals
import os

# Load CSV file
csv_filename = "input.csv"
df = pd.read_csv(csv_filename)

# Define model names
model_names = [
    'access',
    'keep_it_simple',
    'swipe_v5',
    'swipe_v5_clean',
    'gpt3',
    'bartl_wikilarge'
]

# Set sample size for testing
df_sample = df.copy()

# Initialize the Referee model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DebertaForEval(uglobals.DERBERTA_MODEL_DIR, uglobals.DERBERTA_TOKENIZER_DIR, device, head_type='linear')
model.load_state_dict(torch.load("pretrained_deberta.bin", map_location=device)['model_state_dict'], strict=False)
model.eval()

# Function to compute Referee scores
def compute_referee_score(pairs):
    scores = []
    for pair in pairs:
        model_input = [str(pair[0]) + ' ' + model.tokenizer.sep_token + ' ' + str(pair[1])]
        model_out = model(model_input)
        scores.append(model_out[:, -1].item())  # Extract Referee score
    return scores

# Compute Referee scores for each model
source_texts = df_sample['r_content'].tolist()
for name in model_names:
    hypotheses = df_sample['candidate_' + name].tolist()

    # Create input pairs [[source, hypothesis], ...]
    input_pairs = list(zip(source_texts, hypotheses))

    # Compute scores
    df_sample.loc[:, "referee_score_" + name] = compute_referee_score(input_pairs)
# Compute Referee scores for reference texts
reference_texts = df_sample['s_content'].tolist()
reference_pairs = list(zip(source_texts, reference_texts))
df_sample.loc[:, "referee_score_reference"] = compute_referee_score(reference_pairs)

# Save the updated sample CSV file
new_csv_filename = "referee.csv"
df_sample.to_csv(new_csv_filename, index=False)

print(f"CSV file has been updated and saved as: {new_csv_filename}")


Sample CSV file has been updated and saved as: test_model_responses_referee.csv
